In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=2, stride=2, padding=0),  
            nn.ELU(),
            nn.Conv2d(32, 32, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=2, padding=0), 
            nn.ELU(),
            nn.Conv2d(64, 128, kernel_size=2, stride=2, padding=0),
            nn.ELU(), 
            nn.Conv2d(128, 256, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Conv2d(256, 512, kernel_size=2, stride=2, padding=0),
            nn.ELU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AngelaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.mlp = nn.Sequential(
            nn.Linear(features_dim, 128),
            nn.ELU(),
            nn.Linear(128, 128),
            nn.ELU(),
            nn.Linear(128, 128),
            nn.ELU(),
            nn.Linear(128, 128),
            nn.ELU(),
            nn.Linear(128, 128),
            nn.ELU(),
        )
        self.action_head = nn.Linear(128, action_space.n)
        self.value_head = nn.Linear(128, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.mlp.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
        
        with torch.no_grad():
            features = self.encoder(obs)
        features = self.mlp(features)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C32']))
    env = PovOnlyWrapper(env)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C32 pretrained (AngelaCNN + MLP) (3 noops after placement)"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-10-09 14:51:38,435	INFO wandb.py:170 -- Already logged into W&B.
2021-10-09 14:51:38,449	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc
PPO_my_env_67f57_00000,RUNNING,


wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=202262) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=202262)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=202262) 2021-10-09 14:51:41,877	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=202262) 2021-10-09 14:51:41,877	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=202261) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=202261)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=202262) 2021-10-09 14:51:47,687	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=202261) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=202261)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=202261) /root/miniconda/envs/py37/lib/py

Result for PPO_my_env_67f57_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-10-09_14-53-07
  done: false
  episode_len_mean: 424.0
  episode_media: {}
  episode_reward_max: -7.0
  episode_reward_mean: -8.5
  episode_reward_min: -10.0
  episodes_this_iter: 2
  episodes_total: 2
  experiment_id: 9188d825d2554895b0ca0fd85ffa2a7a
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 2.880436126391093
          entropy_coeff: 0.009999999999999998
          kl: 0.004079058113112211
          policy_loss: 0.03850563491384188
          total_loss: 0.6827736003531351
          vf_explained_var: 0.07799533754587173
          vf_loss: 0.672256518734826
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iterations_since_restore: 1
  node_ip: 192.168.3.5
  n

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_67f57_00000,RUNNING,192.168.3.5:202262,646,14949.1,646000,0,0,0,317.49


Result for PPO_my_env_67f57_00000:
  agent_timesteps_total: 647000
  custom_metrics: {}
  date: 2021-10-09_19-01-28
  done: false
  episode_len_mean: 321.84
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 3
  episodes_total: 1675
  experiment_id: 9188d825d2554895b0ca0fd85ffa2a7a
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.1710523009300232
          cur_lr: 5.000000000000001e-05
          entropy: 1.6703581796752083
          entropy_coeff: 0.009999999999999998
          kl: 0.012038912760173975
          policy_loss: -0.06935657180017896
          total_loss: -0.08384120729234483
          vf_explained_var: -0.18715892732143402
          vf_loss: 0.0001596636047502721
    num_agent_steps_sampled: 647000
    num_agent_steps_trained: 647000
    num_steps_sampled: 647000
    num_steps_trained: 647000
  iterations_since_restore: 647
  node

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_67f57_00000,RUNNING,192.168.3.5:202262,647,14968.4,647000,0,0,0,321.84


Result for PPO_my_env_67f57_00000:
  agent_timesteps_total: 648000
  custom_metrics: {}
  date: 2021-10-09_19-01-47
  done: false
  episode_len_mean: 324.0
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 2
  episodes_total: 1677
  experiment_id: 9188d825d2554895b0ca0fd85ffa2a7a
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.1710523009300232
          cur_lr: 5.000000000000001e-05
          entropy: 1.7168709794680277
          entropy_coeff: 0.009999999999999998
          kl: 0.009395367397608743
          policy_loss: -0.06250214498076175
          total_loss: -0.07788888747907347
          vf_explained_var: -0.48133841156959534
          vf_loss: 0.00017487067404241922
    num_agent_steps_sampled: 648000
    num_agent_steps_trained: 648000
    num_steps_sampled: 648000
    num_steps_trained: 648000
  iterations_since_restore: 648
  node

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_67f57_00000,RUNNING,192.168.3.5:202262,648,14987.5,648000,0,0,0,324
